In [1]:
import numpy as np
import pandas as pd
import math
import time

In [2]:
df = pd.read_csv("Desafio ds 2022/ordenamiento.csv")

# Primero algunos sanity checks

In [3]:
print(df.isnull().sum())

item_id     0
vertical    0
category    0
domain      0
score       0
dtype: int64


In [4]:
df.duplicated().sum()

0

# La idea para mi algoritmo se resume de la siguiente manera:

## 1.   De no haber restricciones, solo ordenando la columna de score obtendria el resultado ideal. El hecho de que haya restricciones en general descalifica a este ordenamiento como una solucion satisfactoria, pero no quita el hecho de que sigue siendo una "buena" aproximacion para el resultado restringido. 

## 2.   Voy a pensar el problema como un problema de mapeo de indices. Indices de mi data frame original deben ser mapeados a los indices 0,1,2,.... de un data frame nuevo. Una forma brute force de hacer esto seria tener un double loop  en el cual se chequeen todas las restricciones antes de mapear un indice. Mi approach es en escencia un double loop pero optimizado de distintas formas.

## 3.   Algunos de mis metodos de optimizacion son: llenar previamente algunos indices especiales para evitar if statements en cada loop; reducir el tamanio de los indices del nested loop (indices del data frame original) cada vez que un indice es mapeado ( si ya lo ubique no necesito considerarlo mas!) 

## 4. En conexion con el punto numero uno, realizar el nested loop en un dataframe ordenado de mayor a menor en score ayuda a encontrar al mejor candidato rapidamente

In [5]:
def sort(dfr):
    
    start = time.time()
    
    #Hagamos un data frame vacio
    
    df_final = pd.DataFrame().reindex_like(dfr)
    
    #Hagamos una copia del data frame original donde los cores estes sorteados
    
    df_copy = dfr.copy(deep=True) 
    
    #Primero ordenemos de mayor a menor el score del data frame original
    
    df_copy.sort_values('score',inplace = True,ascending = False)
    
    # La lista de indices del dataframe original con score descendente. Esta lista la vamos a ir
    # podando a medida que los filas vayan siendo colocadas en el dataframe final. 
    # Tambien definimos la lista de indices que nos interesa llenar
    
    index_list = df_copy.index.to_list()
    
    index_list_new = [idx for idx in range(dfr.shape[0])]
    
    #Queremos mantener un registro del ultimo vertical para saber que no hay que reperitlo
    
    vert_dict = {}
    vert_dict['-1'] = np.NaN
    
    #Queremos mantener un registro del ultimo domain_id y un contador que impida que pongamos mas de 3 consecutivos
    
    domain_dict = {}
    domain_dict['-1'] = np.NaN
    domain_counter = 0
    
    # Veamos si los items especiales estan presentes y guardemos sus indices. Tambien voy a removerlos de la lista de
    # indices a ubicar y la lista de indices finales (pues ya sabemos donde los queremos tener)
    
    try:
        esp_1 = dfr.loc[dfr.item_id ==  641416750].index[0]
        index_list.remove(esp_1)
        index_list_new.remove(3)
        vert_dict['3'] = dfr.iloc[esp_1]['vertical']
        domain_dict['3'] = dfr.iloc[esp_1]['domain']
    except:
        esp_1 = np.nan
        print('item_id 641416750 no esta presente')

    try:
        esp_2 = dfr.loc[dfr.item_id ==  22351223].index[0]
        index_list.remove(esp_2)
        index_list_new.remove(6)
        vert_dict['6'] = dfr.iloc[esp_2]['vertical']
        domain_dict['6'] = dfr.iloc[esp_2]['domain']
    except:
        esp_2 = np.nan
        print('item_id 22351223 no esta presente')
    
    #ya podemos ubicar estos elementos donde los necesitamos 
    
    if not math.isnan(esp_1):
        df_final.iloc[3] = dfr.iloc[esp_1]
    if not math.isnan(esp_2):
        df_final.iloc[6] = dfr.iloc[esp_2]
                    
    # Primero llenemos los espacios 9,10,11 con los items de categoria “HOME&DECO” de score mas alto. 

    index_HD = (df_copy.loc[df_copy.category ==  'HOME&DECOR']).index.to_list()
    
    if len(index_HD) <3 :
        raise Exception("No hay suficientes items de categoria HOME&DECOR para satisfacer las restricciones")
    else:
        df_final.iloc[9]  = dfr.iloc[index_HD[0]]
        df_final.iloc[10] = dfr.iloc[index_HD[1]]
        df_final.iloc[11] = dfr.iloc[index_HD[2]]
        
        #Removemos ahora los indices de los cuales ya nos hemos encargado
        index_list_new.remove(9)
        index_list_new.remove(10)
        index_list_new.remove(11)
        index_list.remove(index_HD[0])
        index_list.remove(index_HD[1])
        index_list.remove(index_HD[2])
        
        vert_dict['9'] = dfr.iloc[index_HD[0]]['vertical']
        vert_dict['10'] = dfr.iloc[index_HD[1]]['vertical']
        vert_dict['11'] = dfr.iloc[index_HD[2]]['vertical']
        domain_dict['9'] = dfr.iloc[index_HD[0]]['domain']
        domain_dict['10'] = dfr.iloc[index_HD[1]]['domain']
        domain_dict['11'] = dfr.iloc[index_HD[2]]['domain']
        

        
    # Llenemos el data frame nuevo con los elementos y siguiendo las restricciones! 
    
    for i in index_list_new:
        
        j_index = np.NaN
            
        for j in index_list:
                  
            if (dfr.iloc[j]['vertical'] != vert_dict[str(i-1)]): 
                
                #Hemos cumplido la restriccion de verticals
                
                # Chequeamos si repetimos domain y si el maximo de domains consecutivos es respetado.
                # Updateamos el counter y el domain_prev para que en la siguiente iteracion podamos hacer
                # el mismo chequeo
                
                if dfr.iloc[j]['domain'] == domain_dict[str(i-1)]: 
                    if domain_counter == 3:
                        continue
                    else:
                        #Podemos agregar a la lista
                        df_final.iloc[i] = dfr.iloc[j]
                        domain_counter += 1 
                        vert_dict[str(i)] = dfr.iloc[j]['vertical']
                        j_index = j
                        break
                else:
                    df_final.iloc[i] = dfr.iloc[j]
                    domain_dict[str(i)] = dfr.iloc[j]['domain']
                    domain_counter = 1
                    df_final.iloc[i] = dfr.iloc[j]
                    vert_dict[str(i)] = dfr.iloc[j]['vertical']
                    j_index = j
                    break
                    
            else:            
                continue
                
        
        
        if math.isnan(j_index):
            print('No se ha podido encontrar un item para la posicion '+str(i))
        else:
            #No necesitamos tener en cuenta el item que ya hemos ordenado
            index_list.remove(j_index)
        continue
        
    end = time.time()
    
    print( "Algoritmo tardo "+str(end-start)+" segundos en ordenar la lista")
        
    return df_final
        
                
            
                
            

In [6]:
df

,item_id,vertical,category,domain,score
0,512208310,CPG,PETS FOOD,MLC-CATS_AND_DOGS_FOODS,0.0272
1,468513076,CE,ELECTRONICS,MLC-RANGES,0.9256
2,614337410,CE,ELECTRONICS,MLC-SMART_SPEAKERS,0.8304
3,634351318,APP & SPORTS,APPAREL,MLC-PANTS,0.0560
4,528383704,ACC,VEHICULAR MULTIMEDIA,MLC-GPS,0.2334
...,...,...,...,...,...
4995,621100365,HOME & INDUSTRY,HOME&DECOR,MLC-HOME_LIGHTING_SUPPLIES,0.8520
4996,613555182,BEAUTY & HEALTH,PHARMACEUTICS,MLC-HUMIDIFIERS_AND_VAPORIZERS,0.9298
4997,535350054,HOME & INDUSTRY,STATIONARY,MLC-PENCIL_CASES,0.7832
4998,549193472,HOME & INDUSTRY,HOME&DECOR,MLC-CHRISTMAS_LIGHTS,0.2830


In [7]:
df.sort_values('score',ascending = False)

,item_id,vertical,category,domain,score
2284,590602034,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9998
4793,609438042,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9996
2898,634352041,CE,MOBILE,MLC-CELLPHONES,0.9994
661,615879515,CE,MOBILE,MLC-CELLPHONES,0.9992
2725,631654974,CE,MOBILE,MLC-CELLPHONES,0.9990
...,...,...,...,...,...
3375,911699995,HOME & INDUSTRY,TOOLS AND CONSTRUCTION,MLC-ELECTRIC_PRESSURE_WASHERS,0.0008
1683,539428631,CPG,FOODS,MLC-EDIBLE_PAPERS,0.0006
2422,489295848,APP & SPORTS,APPAREL,MLC-PANTS,0.0004
3894,489817797,CPG,PERSONAL HYGIENE,MLC-WET_BABY_WIPES,0.0002


In [8]:
df_sorted = sort(df)

item_id 22351223 no esta presente
Algoritmo tardo 7.684037446975708 segundos en ordenar la lista


In [9]:
df_sorted[:20]

,item_id,vertical,category,domain,score
0,590602034.0,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9998
1,523534468.0,BEAUTY & HEALTH,PHARMACEUTICS,MLC-SURGICAL_AND_INDUSTRIAL_MASKS,0.9976
2,609438042.0,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9996
3,641416750.0,CE,MOBILE,MLC-CELLPHONES,0.9986
4,541283090.0,HOME & INDUSTRY,TOOLS AND CONSTRUCTION,MLC-ELECTRIC_DRILLS,0.9968
5,634352041.0,CE,MOBILE,MLC-CELLPHONES,0.9994
6,610865341.0,HOME & INDUSTRY,INDUSTRY,MLC-POINTS_OF_SALE_KITS,0.9960
7,615879515.0,CE,MOBILE,MLC-CELLPHONES,0.9992
8,631987792.0,HOME & INDUSTRY,TOOLS AND CONSTRUCTION,MLC-WELDING_MACHINES,0.9944
9,582188629.0,HOME & INDUSTRY,HOME&DECOR,MLC-INDOOR_CURTAINS_AND_BLINDS,0.9822
